In [10]:
import os
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import io
from skimage.transform import resize
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [11]:
file_dir = 'D:\pedestrian\pedestrians128x64'
for i in range(5):
    filename = "%s\per0010%d.ppm" % (file_dir, i)

In [26]:
random.seed(42)
X_pos = []
for i in random.sample(range(900), 400):  ## 随机采样
    filename = "%s\per%05d.ppm" % (file_dir, i)
    try:
        img = io.imread(filename)
    except FileNotFoundError :
        continue
    X_pos.append(hog(image=img))
X_pos = np.array(X_pos)
X_pos_size = len(X_pos)
print(np.array(X_pos).shape)
y = np.ones(X_pos_size)

(399, 6804)


In [27]:
h = 128
w = 64
neg_file_dir = 'D:\pedestrian\pedestrians_neg'
X_neg = []
for i in range(5):
    rad_y = random.randint(0, 512 - h)
    rad_x = random.randint(0, 512 - w)
    for neg_file in os.listdir(neg_file_dir):
        neg_image = resize(io.imread('%s\%s' % (neg_file_dir, neg_file)), (512, 512))[rad_y: rad_y + h,
                    rad_x:rad_x + w]
        X_neg.append(hog(image=neg_image))
X_neg = np.array(X_neg)
# print(X_pos.shape, X_neg.shape)
X = np.concatenate((X_pos, X_neg))
y = np.append(y, np.zeros(len(X_neg)))
print(X.shape)
print(y.shape)

(649, 6804)
(649,)


In [28]:
# X = X.reshape((449, 1980))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(X_train.shape, y_train.shape)

(434, 6804) (434,)


In [29]:
def svm_train(xt, yt):
    result = SVC(kernel='rbf')
    result.fit(xt, yt)
    return result


def svm_score(svm, xt, yt, xts, yts):
    return svm.score(xt, yt), svm.score(xts, yts)

In [30]:
svm = svm_train(X_train, y_train)
score = svm_score(svm, X_train, y_train, X_test, y_test)
print(score)


(0.8940092165898618, 0.8558139534883721)


In [31]:
train_score = []
test_score = []
for i in range(3):
    y_predict = svm.predict(X_train)